In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit, train_test_split
from sklearn.metrics import mean_squared_error

In [30]:
national_demand = pd.read_csv('../data/demand_lower_48')

In [31]:
def drop_utc(string):
    return string[:-6]

In [32]:
def clean_energy_demand_data(df):
    df['Time'] = df['Time'].apply(lambda x: drop_utc(x))
    df['Time'] = pd.to_datetime(df['Time'], errors='coerce')
    df = df.loc[::-1]
    df = df[1:]
    df['Year'] = df['Time'].dt.year
    df['Month'] = df['Time'].dt.month
    df['Hour'] = df['Time'].dt.hour
    df['Day_of_week'] = df['Time'].dt.dayofweek
    df['Day_of_month'] = df['Time'].dt.day
    df['Day_of_year'] = df['Time'].dt.dayofyear
    df['Week_of_year'] = df['Time'].dt.isocalendar().week
    df.set_index('Time', inplace=True, drop=True)
    df = df.astype('int')
    return df

In [33]:
national_demand = clean_energy_demand_data(national_demand)

In [34]:
national_demand.head()

,Megawatthours,Year,Month,Hour,Day_of_week,Day_of_month,Day_of_year,Week_of_year
Time,,,,,,,,
2015-07-01 02:00:00,335153,2015,7,2,2,1,182,27
2015-07-01 03:00:00,333837,2015,7,3,2,1,182,27
2015-07-01 04:00:00,398386,2015,7,4,2,1,182,27
2015-07-01 05:00:00,388954,2015,7,5,2,1,182,27
2015-07-01 06:00:00,392487,2015,7,6,2,1,182,27


In [35]:
def split_data(df, target='Megawatthours', split_date=None):
    X = df
    y = df.pop(target)
    if split_date:
        X_train = X.loc[:split_date]
        X_test = X.loc[split_date:]
        y_train = y.loc[:split_date]
        y_test = y.loc[split_date:]
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y)
    return X_train, X_test, y_train, y_test


    

In [36]:
X_train, X_test, y_train, y_test = split_data(national_demand, split_date='2020-07-01 00:00:00')

In [37]:
xgbr = XGBRegressor()

In [62]:
def grid_search_model(X, y, model, grid, cv_splits=5, metric='neg_root_mean_squared_error'):
    grid_search = GridSearchCV(model, 
                            grid, 
                            cv=TimeSeriesSplit(n_splits=cv_splits),
                            n_jobs=-1,
                            verbose=True,
                            scoring=metric)
    grid_search.fit(X,y)
    return grid_search.best_params_

In [90]:
grid = {'learning_rate': [0.01, 0.1],
 'max_depth': [2],
 'max_features': ['sqrt'],
 'min_samples_leaf': [2],
 'n_estimators': [1500]}

In [91]:
best_params = grid_search_model(X_train, y_train, xgbr, grid)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:   30.0s remaining:   20.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   38.6s finished


[21:37:14] WARNING: ../src/learner.cc:541: 
Parameters: { max_features, min_samples_leaf } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [92]:
best_model = XGBRegressor(**best_params)

In [93]:
best_model

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.1, max_delta_step=None, max_depth=2,
             max_features='sqrt', min_child_weight=None, min_samples_leaf=2,
             missing=nan, monotone_constraints=None, n_estimators=1500,
             n_jobs=None, num_parallel_tree=None, random_state=None,
             reg_alpha=None, reg_lambda=None, scale_pos_weight=None,
             subsample=None, tree_method=None, validate_parameters=None,
             verbosity=None)

In [94]:
best_model.fit(X_train, y_train)

[21:37:30] WARNING: ../src/learner.cc:541: 
Parameters: { max_features, min_samples_leaf } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=2,
             max_features='sqrt', min_child_weight=1, min_samples_leaf=2,
             missing=nan, monotone_constraints='()', n_estimators=1500,
             n_jobs=8, num_parallel_tree=1, random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [95]:
predictions = best_model.predict(X_test)

In [96]:
rmse = np.sqrt(mean_squared_error(y_test, predictions))
rmse

29355.47183339139

In [86]:
from xgboost import plot_importance, plot_tree